In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
aspects = [
    'hotel#general', 'hotel#prices', 'hotel#design&features', 'hotel#cleanliness', 'hotel#comfort', 'hotel#quality', 'hotel#miscellaneous',
    'rooms#general', 'rooms#prices', 'rooms#design&features', 'rooms#cleanliness', 'rooms#comfort', 'rooms#quality', 'rooms#miscellaneous',
    'room_amenities#general', 'room_amenities#prices', 'room_amenities#design&features', 'room_amenities#cleanliness', 'room_amenities#comfort', 'room_amenities#quality', 'room_amenities#miscellaneous',
    'facilities#general', 'facilities#prices', 'facilities#design&features', 'facilities#cleanliness', 'facilities#comfort', 'facilities#quality', 'facilities#miscellaneous',
    'service#general',
    'location#general',
    'food&drinks#prices', 'food&drinks#quality', 'food&drinks#style&options', 'food&drinks#miscellaneous' ]

mapping = {np.nan: 0, 'dne': 0, 'positive': 1, 'negative': 2, 'neutral': 3}
rev_mapping = {1: 'positive', 2: 'negative', 3: 'neutral', 0: np.nan}

# Format txt -> csv

In [3]:
def label_encoder(label):
    y = [0] * len(aspects)
    ap_stm = re.findall('{(.+?), ([a-z]+)}', label)

    for aspect, sentiment in ap_stm:
        idx = aspects.index(aspect)
        y[idx] = mapping.get(sentiment, 0)

    return y

def txt2df(filepath):
    with open(filepath, 'r', encoding='utf-8-sig') as txt:
        data = txt.read().split('\n')

    df = pd.DataFrame()
    df['review'] = [review for review in data[1::4]]
    df[aspects] = [label_encoder(label) for label in data[2::4]]

    return df

In [ ]:
data = txt2df("D:\\21. Học máy thống kê/Đồ án/First_annotate/annotated_reviews.txt")

In [ ]:
# data.to_csv("../Data/data.csv", index=False)

# Format csv -> txt

In [19]:
def label_decoder(encoded_label):
    aps_stms = (pd.to_numeric(encoded_label, errors='coerce')
              .replace(0, np.nan)
              .dropna()
              .astype(int))

    return ', '.join([f'{{{aspect}, {rev_mapping[sentiment]}}}' 
                      for aspect, sentiment in 
                      zip(aps_stms.index, aps_stms)])

def csv2txt(filepath):
    df = pd.read_csv(filepath)
    rows = []
    for id, row in df.iterrows():
        review = row.iloc[0]
        labels = label_decoder(row.iloc[1:])
        rows.extend((f'#{id+1}', review, labels, ''))
    return '\n'.join(rows)

In [ ]:
# output_string = csv2txt("../Data/data.csv")
# output_filepath = '../output_reviews.txt'

# with open(output_filepath, 'w', encoding='utf-8') as f:
#     f.write(output_string)

# print(f"TXT successfully saved to {output_filepath}")

TXT successfully saved to ../output_reviews.txt
